<a href="https://colab.research.google.com/github/wzljerry/Hierarchical-Federated-Learning/blob/main/FL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
'''
@Title : Traditional Federated Learning
@Author : Zhilin Wang
@Email : wangzhil.edu
@Date : 3-08-2022
@Reference: https://towardsdatascience.com/federated-learning-a-step-by-step-implementation-in-tensorflow-aac568283399 
'''
import tensorflow as tf
tf.test.gpu_device_name()
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

import hashlib
import pandas as pd
import numpy as np
from sklearn import datasets
from keras.datasets import mnist
import keras
import random
import time
from random import choice
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import backend as Ks
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from keras.layers import Convolution2D
from tensorflow.keras.layers import Dropout

# to get the training data, and split the data via the number of clients
class Get_data:
  def __init__(self,n):
    self.n=n # number of clients

  def load_data(self):
    (X_train, y_train), (X_test, y_test) = mnist.load_data()
    return X_train, y_train, X_test, y_test

  def flatten_data(self,X):
    data=[]
    for i in X:
      da=i.flatten()
      data.append(da)
    return data

  def one_d_to_n_d(self,X):
    data=[]
    for i in X:
      da=i.reshape(28,28)
      data.append(da)
    return data

  def non_iid(self,X,y):
    train_X=self.flatten_data(X)
    train_y=list(y)
    train_data=np.c_[train_X,train_y]
    sort_data=train_data[np.argsort(train_data[:,784])]
    train_x=sort_data[:,0:784]
    train_Y=np.array(sort_data[:,784])
    train_x_da=np.array(self.one_d_to_n_d(train_x))
    return train_x_da,train_Y

  def split_data(self, data, n): 
    size=int(len(data) / self.n)
    s_data = []
    for i in range(0, int(len(data)) + 1, size):
        c_data = data[i:i + size]
        if c_data != []:
            s_data.append(c_data)
    return s_data

  def pre_data(self):

    X_train, y_train, X_test, y_test=self.load_data()#iid

    X_train,y_train=self.non_iid(X_train,y_train) # non_iid

    X_train=self.split_data(X_train,self.n) 
    y_train=self.split_data(y_train,self.n)
    return X_train, y_train, X_test, y_test

class Model:
  
  def __init__(self):
    pass

  def global_models(self):
    model= tf.keras.models.Sequential([
          tf.keras.layers.Flatten(input_shape=(28, 28)),
          tf.keras.layers.Dense(128, activation='relu'),
          tf.keras.layers.Dropout(0.2),
          tf.keras.layers.Dense(10, activation='softmax')
      ])
    return model
  
  def global_model(self):
    model = Sequential()

    model.add(Convolution2D(32, (3, 3), activation='relu', input_shape=(28,28,1))) 
    model.add(Convolution2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))
    return model

  def evaluate_model(self,model,test_X, test_y):
    model.compile(optimizer='sgd',
                  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
                  metrics=['accuracy'])
    score=model.evaluate(test_X, test_y)
    print("Test Loss:", score[0])
    print('Test Accuracy:', score[1])
    return score[0],score[1]

class Client:
  
  def __init__(self,lr,epoch):
    #self.n=n
    self.epoch=epoch
    self.lr = lr
    self.loss='categorical_crossentropy'
    self.metrics = ['accuracy']
    self.optimizer = SGD(lr=self.lr, 
                decay=self.lr / 2, 
                momentum=0.9
               )
  def weight_client(self,data,m,n):
    wei_client = []
    for i in range(n):
        len_data = len(data[i])
        proba = len_data / m
        wei_client.append(proba)
    return wei_client
    
  
  def scale_model_weights(self,weight,scalar,num):
    '''function for scaling a models weights'''
    weight_final = []
    steps = len(weight)

    fac=scalar[num]

    sca=[fac for i in range(steps)]

    for i in range(steps):
      weight_final.append(sca[i]*weight[i])
        #weight_final.append(weight[i])
    return weight_final

  def training(self,X,y,global_weights):

    #fact=self.weight_client()
    model=Model().global_model()

    model.compile(optimizer=self.optimizer,
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
    
    model.set_weights(global_weights)
    model.fit(X,y,epochs=self.epoch)
    weights=model.get_weights()

    return weights

class Server:
  
  def __init__(self):
    pass
  
  def scale_model_weights(self,weight,scalar,num):
    '''function for scaling a models weights'''
    weight_final = []
    wei=[]
    steps = 8

    for j in range(num):
      fa=scalar[j]
      scar=[fa for k in range(steps)]
      for i in range(steps):
        weight_final.append(scar[i]*weight[j][i])
        #weight_final.append(weight[i])
      wei.append(weight_final)
    return wei

  def sum_scaled_weights(self,scaled_weight_list):
    '''Return the sum of the listed scaled weights. The is equivalent to scaled avg of the weights'''
    avg_grad = list()
    #get the average grad accross all client gradients
    for grad_list_tuple in zip(*scaled_weight_list):
        layer_mean = tf.math.reduce_sum(grad_list_tuple, axis=0)
        avg_grad.append(layer_mean)
    return avg_grad

  def evaluate(self,model,test_X, test_y):
    loss,acc=Model.evaluate_model(model,test_X, test_y)
    return loss,acc

def main(num_client,K,lr,epoch):
  client=Client(lr,epoch)
  get_data=Get_data(num_client)
  train_X,train_y, test_X, test_y=get_data.load_data()
  m=len(train_X)

  X_train, y_train, X_test, y_test=get_data.pre_data()

  server=Server()
  global_model=Model().global_model()

  accuracy=[]
  losses=[]
  factor=[0.1 for i in range(10)]
  for k in range(K):
    print(k)
    global_weights=global_model.get_weights()
    weit=[]
    weitt=[]
    for i in range(num_client):
      weix=client.training(X_train[i], y_train[i],global_weights)
      weix=client.scale_model_weights(weix,factor,i)
      weit.append(weix)
    global_weight=server.sum_scaled_weights(weit) # fedavg
    global_model.set_weights(global_weight)
    loss,acc=Model().evaluate_model(global_model,test_X,test_y)
    losses.append(loss)
    accuracy.append(acc)
  return losses,accuracy

if __name__=='__main__':
  K=100 # number of local rounds
  num_client=10# number of clients for each server
  lr=0.001 # learning rate
  epoch=1 # local iterations

  loss,acc=main(num_client,K,lr,epoch)
  loss
  print('====================================loss================================')
  print(loss)
  print('====================================acc================================')
  print(acc)